In [19]:
import os
import sys
sys.path.append(os.getcwd())
sys.path.append(os.getcwd() + '/../src')


In [20]:
import logging
import wandb
from concurrent.futures import ThreadPoolExecutor
import multiprocessing


In [21]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler('download.log')
    ]
)
logger = logging.getLogger(__name__)

# Set multiprocessing start method to 'spawn'
multiprocessing.set_start_method("spawn", force=True)

In [22]:
# Configuration
download_dir = "/scratch/downloaded_artifacts"
entity = "wlp9800-new-york-university"
project_name = "oho_exps"
group_name = "fixed_weight_seed-1_cf21e420f71a4529bed03b4c48fda84c"
max_download_workers = 20

In [23]:
# Ensure download directory exists
os.makedirs(download_dir, exist_ok=True)

In [24]:
# Function to download a single run's artifact
def download_artifact(run_data):
    run_id = run_data["id"]
    config = run_data["config"]
    try:
        api = wandb.Api()
        artifact = api.artifact(f'{entity}/{project_name}/logs_{run_id}:v0')
        artifact_dir = os.path.join(download_dir, artifact.name)
        artifact.download(root=artifact_dir)
        logger.info(f"Downloaded {artifact.name} to {artifact_dir}")
        return {
            "run_id": run_id,
            "artifact_dir": artifact_dir,
            "config": config,
            "status": "success"
        }
    except Exception as e:
        logger.error(f"Error downloading artifact for run {run_id}: {str(e)}")
        return {
            "run_id": run_id,
            "artifact_dir": None,
            "config": config,
            "status": f"error: {str(e)}"
        }

In [25]:
api = wandb.Api()
runs = api.runs(
    path=f"{entity}/{project_name}",
    filters={"group": group_name}
)

# Prepare run data
run_data = [{"id": run.id, "config": run.config} for run in runs]
logger.info(f"Found {len(run_data)} runs to download")


2025-04-20 15:04:03,424 - INFO - Found 40 runs to download


In [26]:
with ThreadPoolExecutor(max_workers=max_download_workers) as executor:
    download_results = list(executor.map(download_artifact, run_data))

wandb:   1 of 1 files downloaded.  


2025-04-20 15:04:04,834 - INFO - Downloaded logs_u4nlwnuv:v0 to /scratch/downloaded_artifacts/logs_u4nlwnuv:v0


wandb: WARNING Cache size exceeded. Attempting to reclaim space...
wandb: WARNING Cache contains 305.5MB of temporary files. Run `wandb artifact cleanup --remove-temp` to remove them.
wandb:   1 of 1 files downloaded.  


2025-04-20 15:04:06,128 - INFO - Downloaded logs_vi4afo9g:v0 to /scratch/downloaded_artifacts/logs_vi4afo9g:v0


wandb:   1 of 1 files downloaded.  


...

2025-04-20 15:04:09,009 - INFO - Downloaded logs_lgc3gwab:v0 to /scratch/downloaded_artifacts/logs_lgc3gwab:v0


wandb:   1 of 1 files downloaded.  


2025-04-20 15:04:09,011 - INFO - Downloaded logs_a6fk8oxn:v0 to /scratch/downloaded_artifacts/logs_a6fk8oxn:v0


wandb:   1 of 1 files downloaded.  


2025-04-20 15:04:09,020 - INFO - Downloaded logs_qludfphl:v0 to /scratch/downloaded_artifacts/logs_qludfphl:v0


In [27]:
# Save download results for the processing script, including group name
import pickle
results_file = os.path.join(download_dir, f'download_results_{group_name}.pkl')
with open(results_file, 'wb') as f:
    pickle.dump(download_results, f)
logger.info(f"Saved download results to {results_file}")

2025-04-20 15:04:09,075 - INFO - Saved download results to /scratch/downloaded_artifacts/download_results_fixed_weight_seed-1_cf21e420f71a4529bed03b4c48fda84c.pkl
